# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [1]:
#!pip install PyMySQL

### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [10]:
db_name = "database-1"
username = "admin"
password = "admin2021"
host = "database-1.ckazj8ut3bmc.us-east-2.rds.amazonaws.com"
port = 3306

### Conexion BD

In [11]:
import pymysql

In [12]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''
db = pymysql.connect(host=host,
                     user=username,
                     password=password,
                     cursorclass=pymysql.cursors.DictCursor)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [13]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries
de insert de datos y esas cosas...
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL instance version: {version}')

MySQL instance version: {'VERSION()': '8.0.23'}


### Creación de DB

In [7]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL  se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
delete_db = '''DROP DATABASE country_database'''
cursor.execute(delete_db)

# La volvemos a crear
cursor.execute(create_db)
# el output es el numero de filas afectadas

1

In [14]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

### Creación de tablas

In [17]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''


# crear una tabla que se llame country
create_table = '''
CREATE TABLE country (
id INT NOT NULL auto_increment,
name TEXT,
continent TEXT,
population INT,
gdp DOUBLE,
primary key (id)
)
'''

cursor.execute(create_table)

0

### Seleccionar la BD

In [16]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE country_database'''
cursor.execute(use_db)

0

In [18]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_country_database': 'country'}]

In [ ]:
"""
Si queremos borrar alguna tabla

drop_table = '''DROP TABLE country'''
cursor.execute(drop_table)

"""

### Insertar datos

In [19]:
insert_data = '''
INSERT INTO country (name, continent, population, gdp)\
                    values('%s', '%s', '%s', '%s')''' % ('Spain', 'Europe', 47, 1.4)

cursor.execute(insert_data)


1

### Leer datos

In [20]:
sql = '''SELECT * FROM country'''
cursor.execute(sql)

1

In [21]:
cursor.fetchall()

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.4}]

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [22]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [23]:
try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    cursor.fetchall()
except Exception as e:
    print(e)

### Insertar datos de un CSV

In [24]:
# Creamos el CSV
import pandas as pd

df = pd.DataFrame({
    'name': ['France', 'Japan', 'Argentina', 'Brasil'],
    'continent': ['Europe', 'Asia', 'America', 'America'],
    'population': [70, 126, 44, 209],
    'gdp': [2.7, 4.9, 0.5, 1.8]
})

df.to_csv('country_data.csv', sep=';', index=False)

In [25]:
# leemos los datos
df = pd.read_csv('country_data.csv', sep=';')
df.head()

,name,continent,population,gdp
0,France,Europe,70,2.7
1,Japan,Asia,126,4.9
2,Argentina,America,44,0.5
3,Brasil,America,209,1.8


In [26]:
!pip install sqlalchemy --upgrade 

In [28]:
from sqlalchemy import create_engine

# create sqlalchemy engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user=username,
                               host=host,
                               pw=password,
                               db="country_database"))


In [29]:
# insertamos todo el dataframe
df.to_sql('country',
          con= engine,
          if_exists = 'append',
          chunksize = 1000,
          index= False)

In [30]:
db.commit()

In [31]:
# leemos los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM country'''
cursor.execute(sql)
cursor.fetchall()

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.4},
 {'id': 2,
  'name': 'France',
  'continent': 'Europe',
  'population': 70,
  'gdp': 2.7},
 {'id': 3,
  'name': 'Japan',
  'continent': 'Asia',
  'population': 126,
  'gdp': 4.9},
 {'id': 4,
  'name': 'Argentina',
  'continent': 'America',
  'population': 44,
  'gdp': 0.5},
 {'id': 5,
  'name': 'Brasil',
  'continent': 'America',
  'population': 209,
  'gdp': 1.8}]

### Cerrar la conexión cuando acabemos

In [32]:
db.close()